Создание датафрейма с значениями RGB

In [3]:
from pathlib import Path
import pandas as pd
from PIL import Image

import tensorflow as tf
from tensorflow.keras import layers, models

import numpy as np

p = Path('data/images')
list_name_img = []
list_var_arr = []
for x in p.glob("*"):
    x_rprefix = str(x).removeprefix('data\images\\')
    x_rprefix = x_rprefix[:10]
    list_name_img.append(x_rprefix)

    img = Image.open(x)
    img_resize = img.resize((32, 32))
    list_var_arr.append(np.array(img_resize) / 255.0)

list_name_img_df = pd.DataFrame({'id': list_name_img, 'val_arr': list_var_arr})
list_name_img_df.head()

,id,val_arr
0,01a7MiLURN,"[[[0.5882352941176471, 0.36470588235294116, 0...."
1,01SPjdJuM4,"[[[0.6431372549019608, 0.803921568627451, 0.95..."
2,05q5J7VWr4,"[[[0.6862745098039216, 0.7294117647058823, 0.7..."
3,0666kmCAAC,"[[[0.3764705882352941, 0.3686274509803922, 0.3..."
4,07aHL0dVg4,"[[[0.43529411764705883, 0.35294117647058826, 0..."


Датафрейм с тренировочными данными

In [4]:
train_df = pd.read_csv('data/train.csv')
train_df_id = train_df['id'].str[:10]
train_df_target = train_df['target']
train_df_new = pd.concat([train_df_id, train_df_target], axis=1)
train_df_new

,id,target
0,gL5fWrgNPU,0
1,KKWXJDN8uM,1
2,Wb2I0CXlAY,0
3,G7PLhKpaf7,1
4,DEUkeefz6Z,0
...,...,...
1007,fqoVycdxLV,1
1008,TN2ZOiNI5e,0
1009,2WMWt6CAWQ,0
1010,fQCY5n2vcF,0


Объединение двух датафреймов

In [5]:
result_df = pd.merge(train_df_new, list_name_img_df, on='id', how='left')
result_df

,id,target,val_arr
0,gL5fWrgNPU,0,"[[[0.45098039215686275, 0.3764705882352941, 0...."
1,KKWXJDN8uM,1,"[[[0.9254901960784314, 0.9254901960784314, 0.9..."
2,Wb2I0CXlAY,0,"[[[0.13333333333333333, 0.11372549019607843, 0..."
3,G7PLhKpaf7,1,"[[[0.10588235294117647, 0.09411764705882353, 0..."
4,DEUkeefz6Z,0,"[[[0.3333333333333333, 0.39215686274509803, 0...."
...,...,...,...
1007,fqoVycdxLV,1,"[[[0.8588235294117647, 0.9333333333333333, 0.9..."
1008,TN2ZOiNI5e,0,"[[[0.2901960784313726, 0.15294117647058825, 0...."
1009,2WMWt6CAWQ,0,"[[[0.01568627450980392, 0.0196078431372549, 0...."
1010,fQCY5n2vcF,0,"[[[0.24705882352941178, 0.2235294117647059, 0...."


Обучение примитивной CNN-сети

In [6]:
# (train_images, train_labels) = np.array(result_df['val_arr'])[:800], np.array(result_df['target'])[:800]
# (test_images, test_labels) = np.array(result_df['val_arr'])[800:], np.array(result_df['target'])[800:]

In [7]:
train_images = result_df[['val_arr']][:800].to_numpy()
test_images = result_df[['val_arr']][800:].to_numpy()
train_images_new = [train_images[0][0] for _ in range(train_images.shape[0])]
test_images_new = [test_images[0][0] for _ in range(test_images.shape[0])]
train_images_tensor = tf.convert_to_tensor(train_images_new)
test_images_tensor = tf.convert_to_tensor(test_images_new)
train_labels_tensor = tf.convert_to_tensor(np.array(result_df['target'])[:800])
test_labels_tensor = tf.convert_to_tensor(np.array(result_df['target'])[800:])

In [8]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.summary()

c:\mArt\dataScience\kaggle\attempt_at_writing\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 4, 4, 64)       │        36,928 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 56,320 (220.00 KB)

 Trainable params: 56,320 (220.00 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2))

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 4, 4, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        65,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 122,050 (476.76 KB)

 Trainable params: 122,050 (476.76 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['sparse_categorical_crossentropy'])

history = model.fit(train_images_tensor, train_labels_tensor, epochs=20, 
                    validation_data=(test_images_tensor, test_labels_tensor))

test_loss, test_acc = model.evaluate(test_images_tensor,  test_labels_tensor, verbose=2)
print(test_acc)

Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 105ms/step - loss: 0.6582 - sparse_categorical_crossentropy: 4.2273 - val_loss: 0.6301 - val_sparse_categorical_crossentropy: 4.7654
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.6366 - sparse_categorical_crossentropy: 5.1230 - val_loss: 0.6655 - val_sparse_categorical_crossentropy: 4.3860
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - loss: 0.6503 - sparse_categorical_crossentropy: 4.7951 - val_loss: 0.6617 - val_sparse_categorical_crossentropy: 4.4494
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - loss: 0.6604 - sparse_categorical_crossentropy: 5.4288 - val_loss: 0.6438 - val_sparse_categorical_crossentropy: 4.5926
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.6540 - sparse_categorical_crossentropy: 5.2981 - val_loss: 0.6307 - val_sparse_categorical_crossentropy: 4.6747
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - loss: 0.6391 - sparse_categorical_crossentropy: 5.1320 - val_loss: 0.6452 - val_

Обработка тестов и предсказание

In [11]:
test_df = pd.read_csv('data/test.csv')
test_images_df = pd.DataFrame(test_df['id'].str[:10])
test_images_df.shape

(506, 1)

In [12]:
result_df = pd.merge(test_images_df, list_name_img_df, on='id', how='left')
result_df

,id,val_arr
0,Qt1fGUB0Vz,"[[[0.09019607843137255, 0.1411764705882353, 0...."
1,j4Rhioq7R3,"[[[0.3843137254901961, 0.4549019607843137, 0.4..."
2,rD0hgFHJUZ,"[[[0.4588235294117647, 0.42745098039215684, 0...."
3,aY5z1EJsJ6,"[[[0.3058823529411765, 0.5333333333333333, 0.6..."
4,qZ3IoxD2TE,"[[[0.2196078431372549, 0.35294117647058826, 0...."
...,...,...
501,Qrme6EUmL1,"[[[0.6313725490196078, 0.5725490196078431, 0.4..."
502,4369Dl8wfs,"[[[0.4235294117647059, 0.4470588235294118, 0.3..."
503,BrNEvRNpPu,"[[[0.9529411764705882, 0.9686274509803922, 0.9..."
504,DuwXCNmeMx,"[[[0.7490196078431373, 0.4823529411764706, 0.2..."


In [13]:
t_images = result_df[['val_arr']].to_numpy()
t_images_new = [t_images[0][0] for _ in range(t_images.shape[0])]
t_images_tensor = tf.convert_to_tensor(t_images_new)

In [19]:
res_pred = model.predict(t_images_tensor, verbose=2)
res_pred[200]

16/16 - 0s - 15ms/step


array([ 0.33132032, -0.19155696], dtype=float32)